# Data preprocessing for the Zindi/GIZ project

Mount Google drive



In [ ]:
from google.colab import files

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


Install and load libraries

In [ ]:
!pip install -q fastai fastcore --upgrade # Make sure we have the new version
!pip install -q python_speech_features
!pip install python_speech_features

     |████████████████████████████████| 194kB 2.8MB/s 
     |████████████████████████████████| 51kB 4.3MB/s 
     |████████████████████████████████| 12.8MB 329kB/s 
     |████████████████████████████████| 776.7MB 22kB/s 


In [ ]:
import pandas as pd
import IPython.display as ipd
import librosa
import python_speech_features as psf
from matplotlib import pyplot as plt
import numpy as np
from os import listdir
from os.path import isfile, join
import os

Read the data

In [ ]:
# Copy the files in and unzip
# !cp 'drive/My Drive/audio_files.zip' audio_files2.zip
# !unzip -q audio_files2.zip

Load the new "train_full.csv" data

In [ ]:
train_full = pd.read_csv("/content/drive/My Drive/ZINDI/train_full.csv")

# Adjust name
train_full['fn'] = train_full['fn'].str.replace('audio_train/', '/content/drive/My Drive/ZINDI/data/audio_train/')
train_full['fn'] = train_full['fn'].str.replace('audio_files/', '/content/drive/My Drive/ZINDI/data/audio_train/')
train_full.head()

,fn,label
0,/content/drive/My Drive/ZINDI/data/audio_train...,akawuka
1,/content/drive/My Drive/ZINDI/data/audio_train...,banana
2,/content/drive/My Drive/ZINDI/data/audio_train...,obulwadde
3,/content/drive/My Drive/ZINDI/data/audio_train...,nnyaanya
4,/content/drive/My Drive/ZINDI/data/audio_train...,pampu


Load the old train data

In [ ]:
train1 = pd.read_csv("/content/drive/My Drive/ZINDI/Train.csv")

# Adjust name
train1['fn'] = train1['fn'].str.replace('audio_files/', '/content/drive/My Drive/ZINDI/data/audio_train/')
train1.head()

,fn,label
0,/content/drive/My Drive/ZINDI/data/audio_train...,akawuka
1,/content/drive/My Drive/ZINDI/data/audio_train...,banana
2,/content/drive/My Drive/ZINDI/data/audio_train...,obulwadde
3,/content/drive/My Drive/ZINDI/data/audio_train...,nnyaanya
4,/content/drive/My Drive/ZINDI/data/audio_train...,pampu


Add the new train data

In [ ]:
train2 = pd.read_csv('/content/drive/My Drive/ZINDI/labels_new.csv')
train2.head(20)

# Adjust name
train2['fn'] = train2['fn'].str.replace('audio_files2/', '/content/drive/My Drive/ZINDI/data/audio_train/')
train2.head()

,fn,label
0,/content/drive/My Drive/ZINDI/data/audio_train...,abalimi
1,/content/drive/My Drive/ZINDI/data/audio_train...,abalimi
2,/content/drive/My Drive/ZINDI/data/audio_train...,abalimi
3,/content/drive/My Drive/ZINDI/data/audio_train...,abalimi
4,/content/drive/My Drive/ZINDI/data/audio_train...,abalimi


Combine both

In [ ]:
train = train1.append(train2)
train.head()

,fn,label
0,/content/drive/My Drive/ZINDI/data/audio_train...,akawuka
1,/content/drive/My Drive/ZINDI/data/audio_train...,banana
2,/content/drive/My Drive/ZINDI/data/audio_train...,obulwadde
3,/content/drive/My Drive/ZINDI/data/audio_train...,nnyaanya
4,/content/drive/My Drive/ZINDI/data/audio_train...,pampu


Load the test data

In [ ]:
test = pd.read_csv('/content/drive/My Drive/ZINDI/SampleSubmission.csv')
test.head()

# Adjust name
test['fn'] = test['fn'].str.replace('audio_files/', '/content/drive/My Drive/ZINDI/data/audio_test/')
test.head()

,fn,maize streak virus,disease,okukkoola,muwogo,mpeke,mucungwa,greens,garden,mango,bulimi,obuwuka,ebikoola,obulimi,ebisoolisooli,kaamulali,eddagala,beans,omuyembe,leaf,kisaanyi,leaves,butterfly,okuzifuuyira,micungwa,ppaapaali,emboga,kikolo,harvest,olusuku,coffee,super grow,rice,ensujju,okulima,worm,ebbugga,onion,ensigo,plantation,...,ejjobyo,omulimi,okusimba,sweet potatoes,okufuuyira,farming instructor,nnasale beedi,passion fruit,ekitooke,ebisaanyi,ekyeya,enva endiirwa,emisiri,emiyembe,amatooke,ebiwuka,farm,ebinyebwa,amappapaali,ebimera,kassooli,harvesting,emmwanyi,akamonde,obumonde,cabbages,akasaanyi,spread,ebirime,drought,kasaanyi,suckers,insects,fertilizer,nakavundira,ekiwojjolo,akawuka,ddagala,ebiwojjolo,obutungulu
0,/content/drive/My Drive/ZINDI/data/audio_test/...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,/content/drive/My Drive/ZINDI/data/audio_test/...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,/content/drive/My Drive/ZINDI/data/audio_test/...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,/content/drive/My Drive/ZINDI/data/audio_test/...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,/content/drive/My Drive/ZINDI/data/audio_test/...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Get spectrograms

In [ ]:
# Define the function

def im_from_audio(fn, sample_rate=44100, window_length=0.05, window_step=0.0045, NFFT=2205):
  
  # Load the audio into an array (signal) at the specified sample rate
  signal, sr = librosa.load(fn, sr=sample_rate)

  # preemphasis
  signal = psf.sigproc.preemphasis(signal, coeff=0.95)

  # get specrogram
  # Get the frames
  frames = psf.sigproc.framesig(signal, 
                                  window_length*sample_rate, 
                                  window_step*sample_rate, 
                                  lambda x:np.ones((x,)))        # Window function 
    
  # magnitude Spectrogram
  spectrogram = np.rot90(psf.sigproc.magspec(frames, NFFT))
  
  # get rid of high frequencies
  spectrogram = spectrogram[512:,:]

  # normalize in [0, 1]
  spectrogram -= spectrogram.min(axis=None)
  spectrogram /= spectrogram.max(axis=None)        

  # Clip to max 512, 512
  spectrogram = spectrogram[:512, :512]
  
  return spectrogram 


1) Apply the function to the full_train data

In [ ]:
im_from_audio(train_full.fn.sample().values[0])

array([[2.47481438e-05, 3.20675434e-05, 2.32169464e-05, ...,
        2.91702498e-05, 2.28525370e-05, 2.42317890e-05],
       [5.19229184e-05, 5.01928611e-05, 5.82445916e-05, ...,
        1.72388321e-05, 5.01327798e-06, 1.23229928e-06],
       [2.58894934e-05, 2.26804607e-05, 1.57010191e-05, ...,
        2.02921967e-05, 1.99900172e-05, 1.55215957e-05],
       ...,
       [7.79127087e-05, 7.70650847e-05, 7.09717636e-05, ...,
        6.51576797e-05, 6.25693344e-05, 3.95612505e-05],
       [1.18042418e-04, 1.33327380e-04, 1.39966297e-04, ...,
        1.97359049e-04, 1.25001400e-04, 1.56054030e-04],
       [5.24854046e-05, 5.21433321e-05, 4.79042943e-05, ...,
        1.44049083e-04, 1.23932688e-04, 1.12086592e-04]])

In [ ]:
# Load all files and transform them into spectograms
for i in train_full.fn:
  interim1 = i.replace('.wav', '')
  interim2 = interim1.replace('/content/drive/My Drive/ZINDI/data/audio_train/', '') 
  name = '/content/drive/My Drive/ZINDI/data/spectrogram_train/' + interim2 + '-spectrogram.png'
  plt.imsave(fname = name, arr = im_from_audio(i))

Check if it worked:

In [ ]:
# Access the file names in "spectogram-train"
mypath = "/content/drive/My Drive/ZINDI/data/spectrogram_train/"
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

# Check for length
len(files) # 4709 files in the spectogram-train folder

4709

2) Apply the function to the test (validation) data

In [ ]:
# Load all files and transform them into spectograms
for i in test.fn:
  interim1 = i.replace('.wav', '')
  interim2 = interim1.replace('/content/drive/My Drive/ZINDI/data/audio_test/', '') 
  name = '/content/drive/My Drive/ZINDI/data/spectrogram_test/' + interim2 + '-spectrogram.png'
  plt.imsave(fname = name, arr = im_from_audio(i))

Check if it worked:


In [ ]:
# Access the file names in "spectogram-test"
mypath = "/content/drive/My Drive/ZINDI/data/spectrogram_test/"
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

# Check for length
len(files) # 1017 files in the spectogram-test folder

1017

1) Apply the function to the old train data

In [ ]:
##############################################
#### HERE IS WHERE THE PROBLEM STARTED #######
##############################################


# Create directory

# define the name of the directory to be created
# path = "/content/drive/My Drive/ZINDI/data/spectrogram_train/"

# os.mkdir(path)

# Load all files and transform them into spectograms
for i in train_full.fn:
  interim1 = i.replace('.wav', '')
  interim2 = interim1.replace('/content/drive/My Drive/ZINDI/data/audio_train/', '') 
  name = '/content/drive/My Drive/ZINDI/data/spectrogram_train/' + interim2 + '-spectrogram.png'
  plt.imsave(fname = name, arr = im_from_audio(i))

FileNotFoundError: ignored

2) Apply the function to the new train data

In [ ]:
for i in train2.fn:
  interim1 = i.replace('.wav', '')
  interim2 = interim1.replace('audio_files2/', '')
  name = '/content/drive/My Drive/spectogram/' + interim2 + '-spectogram.png'
  plt.imsave(fname = name, arr = im_from_audio(i))

3) Apply the function to the augmented data

In [ ]:
# Load all files and transform them into spectograms
for i in testaugmented.fn:
  interim1 = i.replace('.wav', '')
  interim2 = interim1.replace('audio_files/', '') 
  name = '/content/drive/My Drive/spectogram-test/' + interim2 + '-spectogram.png'
  plt.imsave(fname = name, arr = im_from_audio(i))

We first thought that there were some files missing in the test data spectrograms -- but there weren't. Here's just the code that we used for validation.

In [ ]:
# Check for the missing names in the validation (test) data set

# Access the file names in "spectogram-test"
mypath = "/content/drive/My Drive/spectogram-test"
files = [f for f in os.listdir(mypath) if isfile(join(mypath, f))]

# Check for length
len(files) # 1017 files in the spectogram-test folder

# Load test data
test = pd.read_csv('drive/My Drive/SampleSubmission.csv')

# Check for the names in the validation (test) data set
# 1) Generate interim name where we replace .wav with ''
interim1 = test['fn'].str.replace('.wav', '')
# 2) Generate interim where we then replace "audio_files/" with '' 
# (I couldn't bring it into one single line of code...)
interim2 = interim1.str.replace('audio_files/', '') 
# Combine both interim2 with "spectogram.png" 
# (to make both name similar to "files")
name = interim2 + "-spectogram.png"

# Check for length
len(name) # 1017 names in the validation (test) data set

NameError: ignored